In [26]:
from gnn_tracking.metrics.losses import PotentialLoss
from gnn_tracking.training.callbacks import PrintValidationMetrics
from gnn_tracking.training.tc import TCModule
from gnn_tracking.models.graph_construction import MLGraphConstructionFromChkpt
from gnn_tracking.models.track_condensation_networks import PreTrainedECGraphTCN
import torch

In [9]:

model = PreTrainedECGraphTCN(
      ec=None,
      node_indim= 38,
      edge_indim= 76,
      h_dim= 192,
      e_dim= 192,
      hidden_dim= 192,
      h_outdim= 24,
      L_hc= 5,
      alpha_latent= 0.5,
      n_embedding_coords= 24,
)
preproc = MLGraphConstructionFromChkpt(
    ml_chkpt_path="/home/kl5675/Documents/23/git_sync/hyperparameter_optimization2/scripts/full_detector/lightning_logs/merciful-reindeer-of-coffee/checkpoints/epoch=79-step=72000.compat.ckpt",
    max_num_neighbors= 15,
    max_radius= 1.,
    use_embedding_features= True,
    build_edge_features= True,
)

potential_loss = PotentialLoss(
      q_min= 0.01,
      radius_threshold= 1.0,
      attr_pt_thld= 0.9,
      max_neighbors= 128,
    
)

oc = TCModule(
    model=model,
    preproc=preproc,
    lw_repulsive=0.74,
    lw_background=0,
    background_loss=None,
    potential_loss=potential_loss,
    cluster_scanner=None,
)

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'hc_in' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['hc_in'])`.
  rank_zero_warn(
[20:40:33] DEBUG: Getting class MLModule from module gnn_tracking.training.ml
[20:40:33] DEBUG: Loading checkpoint /home/kl5675/Documents/23/git_sync/hyperparameter_optimization2/scripts/full_detector/lightning_logs/merciful-reindeer-of-coffee/checkpoints/epoch=79-step=72000.compat.ckpt
[20:40:33] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[20:40:33] DEBUG: Getting class GraphConstructionHingeEmbeddingLoss from module gnn_tracking.metrics.losses
[20:40:33] DEBUG: Getting class GraphConstructionKNNScanner from module gnn_tracking.graph_construction.k_scanner
[20:40:33] DEBUG: Checkpoint loaded. Model ready to go.


In [10]:
# import torch
# 
# state_dict = torch.load("/home/kl5675/Documents/23/git_sync/hyperparameter_optimization2/scripts/pixel/lightning_logs/wakeful-premium-partridge/checkpoints/epoch=196-step=177300.ckpt")["state_dict"]


# oc.load_state_dict(
#     ,
#     strict=False,
# )

In [65]:
from gnn_tracking.utils.loading import TrackingDataModule

dm = TrackingDataModule(
    identifier="point_clouds_v10",
    train=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_1/"
        ],
        sample_size=5,
        # If you run into memory issues, reduce this
    ),
    val=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_9/"
        ],
        stop=5
    ),
)


In [66]:
# from pytorch_lightning import Trainer
# 
# trainer = Trainer(
#     callbacks=[PrintValidationMetrics()],
# )
# trainer.fit(oc, dm)

In [67]:
dm.setup("fit")
data = dm.datasets["train"][0]

[20:46:01] INFO: DataLoader will load 900 graphs (out of 900 available).
[20:46:01] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_1/data21999_s0.pt
[20:46:01] INFO: DataLoader will load 5 graphs (out of 1000 available).
[20:46:01] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_9/data29004_s0.pt


In [68]:
data_preproc = preproc(data)

In [15]:
# x = data_preproc.x

In [16]:
# import torch
# 
# torch.isnan(x).any()

In [17]:
# from torch.nn.functional import normalize
# 
# x_normalized = normalize(x, p=2., dim=1, eps=1e-12, out=None)

In [18]:
# x_normalized

In [19]:
# x_normalized.shape

In [20]:
# x_enc = model._gtcn.hc_node_encoder(x_normalized[:100, :])

In [21]:
# torch.isnan(x_enc).sum() / x_enc.numel()

In [22]:
# x_enc

In [ ]:
model_out = model(data_preproc)

In [ ]:
torch.isnan(model_out["H"]).any(), torch.isnan(model_out["B"]).any()

In [1]:
list(model._gtcn.hc_node_encoder.layers.parameters())

NameError: name 'model' is not defined

In [60]:
d = data_preproc
o = model_out
loss_dct = potential_loss(
    beta=o["B"],
    x=o["H"],
    particle_id=d.particle_id,
    reconstructable=d.reconstructable,
    pt=d.pt,
)

In [61]:
loss_dct

{'attractive': tensor(0.0664, grad_fn=<MulBackward0>),
 'repulsive': tensor(0.5165, grad_fn=<MulBackward0>)}

In [62]:
loss = loss_dct["attractive"] + loss_dct["repulsive"]

In [63]:
loss.backward()

In [64]:
list(model._gtcn.hc_node_encoder.parameters())

[Parameter containing:
 tensor([[-0.0913, -0.0627,  0.0372,  ...,  0.0461, -0.0669, -0.0834],
         [ 0.0494, -0.1402,  0.0252,  ...,  0.0044,  0.0239,  0.0807],
         [-0.0353, -0.1025, -0.1559,  ..., -0.1275,  0.0557,  0.0831],
         ...,
         [-0.1032, -0.0065, -0.1365,  ..., -0.1311, -0.0340,  0.0621],
         [-0.1529,  0.0482, -0.1161,  ...,  0.1433, -0.0711,  0.0559],
         [-0.0393,  0.0991,  0.0717,  ...,  0.0068, -0.0254,  0.0553]],
        requires_grad=True),
 Parameter containing:
 tensor([[-0.0570,  0.0624, -0.0361,  ...,  0.0418, -0.0222,  0.0285],
         [-0.0041,  0.0351,  0.0473,  ...,  0.0366, -0.0312,  0.0297],
         [ 0.0488, -0.0041,  0.0441,  ...,  0.0455,  0.0378, -0.0271],
         ...,
         [ 0.0437, -0.0397, -0.0206,  ...,  0.0158,  0.0329,  0.0503],
         [ 0.0226, -0.0669,  0.0128,  ...,  0.0150,  0.0669, -0.0117],
         [-0.0394,  0.0419, -0.0376,  ...,  0.0264, -0.0515,  0.0502]],
        requires_grad=True)]

In [55]:
potential_loss.hparams

"attr_pt_thld":     0.9
"max_neighbors":    128
"q_min":            0.01
"radius_threshold": 1.0